In [1]:
import syft as sy
from syft import ActionObject
from syft import syft_function, syft_function_single_use
from time import sleep
import os
import psutil
import inspect

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft


In [2]:
sy.stage_protocol_changes()

SyftSuccess: 133 Protocol Updates Staged to dev

In [3]:
node = sy.orchestra.launch(name="test-domain-helm2", dev_mode=True,
                           reset=True, 
                           n_consumers=3,
                           create_producer=True,
                           queue_port=3322)
                           
client = node.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Data Migrated to latest version !!!
Logged into <test-domain-helm2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
x = ActionObject.from_obj([1,2])
x_ptr = x.send(client)

In [5]:
@syft_function()
def process_batch(batch):
    # takes 30 hours normally
    print(f"starting batch {batch}")
    from time import sleep
    sleep(1)
    print("done")
    return batch+1

SyftSuccess: Syft function 'process_batch' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [6]:
client.code.submit(process_batch)

SyftSuccess: User Code Submitted

In [7]:
@syft_function()
def middle_middle_job(domain, batch):
    # takes 30 hours normally
    print(f"starting batch {batch}")
    from time import sleep
    sleep(1)
    batch_job = domain.launch_job(process_batch, batch=batch)
    print("start leaf job")
    return 2
    

SyftSuccess: Syft function 'middle_middle_job' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [8]:
client.code.submit(middle_middle_job)

SyftSuccess: User Code Submitted

In [9]:
@syft_function()
def middle_job(domain, batch):
    # takes 30 hours normally
    print(f"starting batch {batch}")
    from time import sleep
    sleep(1)
    batch_job = domain.launch_job(middle_middle_job, batch=batch)
    print("start leaf job")
    return 2
    

SyftSuccess: Syft function 'middle_job' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [10]:
client.code.submit(middle_job)

SyftSuccess: User Code Submitted

In [11]:
@syft_function_single_use(x=x_ptr)
def process_all(domain, x):
    jobs = []
    print("Launching jobs")
    for elem in x:
        batch_job = domain.launch_job(middle_job, batch=elem)
        jobs += [batch_job]
    print("starting aggregation")
    print("Done")
    return 1


SyftSuccess: Syft function 'process_all' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [12]:
req = client.code.request_code_execution(process_all)
req

syft.service.request.request.Request

In [13]:
client.requests[0]

syft.service.request.request.Request

In [14]:
client.requests[0].code

```python
class UserCode
    id: UID = 67823a91dbcc47c4a131fa1c307f9314
    service_func_name: str = process_all
    shareholders: list = ['test-domain-helm2']
    status: list = ['Node: test-domain-helm2, Status: pending']
    
    code:

@syft_function_single_use(x=x_ptr)
def process_all(domain, x):
    jobs = []
    print("Launching jobs")
    for elem in x:
        batch_job = domain.launch_job(middle_job, batch=elem)
        jobs += [batch_job]
    print("starting aggregation")
    print("Done")
    return 1



  Nested Requests:
  class UserCode
      id: UID = d9c59810bea0486095944b54db12b609
      service_func_name: str = middle_job
      shareholders: list = []
      status: list = ['Node: test-domain-helm2, Status: pending']
      
      code:
  
  @syft_function()
  def middle_job(domain, batch):
      # takes 30 hours normally
      print(f"starting batch {batch}")
      from time import sleep
      sleep(1)
      batch_job = domain.launch_job(middle_middle_job, batch=batch)
      print("start leaf job")
      return 2
  
  
  
    Nested Requests:
    class UserCode
        id: UID = 0b70a83dd73c4a2ea839dd735ad7dfab
        service_func_name: str = middle_middle_job
        shareholders: list = []
        status: list = ['Node: test-domain-helm2, Status: pending']
        
        code:
    
    @syft_function()
    def middle_middle_job(domain, batch):
        # takes 30 hours normally
        print(f"starting batch {batch}")
        from time import sleep
        sleep(1)
        batch_job = domain.launch_job(process_batch, batch=batch)
        print("start leaf job")
        return 2
    
    
    
      Nested Requests:
      class UserCode
          id: UID = f4929e1833dd413fb6b5e95d93e77e46
          service_func_name: str = process_batch
          shareholders: list = []
          status: list = ['Node: test-domain-helm2, Status: pending']
          
          code:
      
      @syft_function()
      def process_batch(batch):
          # takes 30 hours normally
          print(f"starting batch {batch}")
          from time import sleep
          sleep(1)
          print("done")
          return batch+1
      
```

In [15]:
client.requests[0].codes

[syft.service.code.user_code.UserCode,
 syft.service.code.user_code.UserCode,
 syft.service.code.user_code.UserCode,
 syft.service.code.user_code.UserCode]

In [16]:
### Deciding if to approve the requests....

In [17]:
client.requests[-1].code

```python
class UserCode
    id: UID = 67823a91dbcc47c4a131fa1c307f9314
    service_func_name: str = process_all
    shareholders: list = ['test-domain-helm2']
    status: list = ['Node: test-domain-helm2, Status: pending']
    
    code:

@syft_function_single_use(x=x_ptr)
def process_all(domain, x):
    jobs = []
    print("Launching jobs")
    for elem in x:
        batch_job = domain.launch_job(middle_job, batch=elem)
        jobs += [batch_job]
    print("starting aggregation")
    print("Done")
    return 1



  Nested Requests:
  class UserCode
      id: UID = d9c59810bea0486095944b54db12b609
      service_func_name: str = middle_job
      shareholders: list = []
      status: list = ['Node: test-domain-helm2, Status: pending']
      
      code:
  
  @syft_function()
  def middle_job(domain, batch):
      # takes 30 hours normally
      print(f"starting batch {batch}")
      from time import sleep
      sleep(1)
      batch_job = domain.launch_job(middle_middle_job, batch=batch)
      print("start leaf job")
      return 2
  
  
  
    Nested Requests:
    class UserCode
        id: UID = 0b70a83dd73c4a2ea839dd735ad7dfab
        service_func_name: str = middle_middle_job
        shareholders: list = []
        status: list = ['Node: test-domain-helm2, Status: pending']
        
        code:
    
    @syft_function()
    def middle_middle_job(domain, batch):
        # takes 30 hours normally
        print(f"starting batch {batch}")
        from time import sleep
        sleep(1)
        batch_job = domain.launch_job(process_batch, batch=batch)
        print("start leaf job")
        return 2
    
    
    
      Nested Requests:
      class UserCode
          id: UID = f4929e1833dd413fb6b5e95d93e77e46
          service_func_name: str = process_batch
          shareholders: list = []
          status: list = ['Node: test-domain-helm2, Status: pending']
          
          code:
      
      @syft_function()
      def process_batch(batch):
          # takes 30 hours normally
          print(f"starting batch {batch}")
          from time import sleep
          sleep(1)
          print("done")
          return batch+1
      
```

In [18]:
client.requests[-1].approve(approve_nested=True)

Request approved for domain test-domain-helm2


SyftSuccess: Request 56f76f67cf544edb8894139a92677a83 changes applied

In [19]:
job = client.code.process_all(x=x_ptr, blocking=False)

In [20]:
job

```python
class Job:
    id: UID = 69b2ab73f729484083e568d37a337a48
    status: created
    has_parent: False
    result: ActionDataEmpty <None>
    logs:

0 
    
```

In [22]:
job.subjobs[0]

```python
class Job:
    id: UID = 5bdd231de10046b69acad09e131ac6e8
    status: completed
    has_parent: True
    result: ActionDataEmpty <None>
    logs:

0 starting batch 2
1 start leaf job
JOB COMPLETED
    
```

24/11/23 17:37:19 FUNCTION LOG (f1aba6d42c3347bda56d0e38ccd45764): starting batch 1
24/11/23 17:37:20 FUNCTION LOG (29d2a2db42954664bc3e65071997f0c4): starting batch 2
24/11/23 17:37:21 FUNCTION LOG (f1aba6d42c3347bda56d0e38ccd45764): start leaf job
24/11/23 17:37:21 FUNCTION LOG (29d2a2db42954664bc3e65071997f0c4): start leaf job
24/11/23 17:37:22 FUNCTION LOG (ff78ff43033642eea84ddf083638e4b6): starting batch 1
24/11/23 17:37:23 FUNCTION LOG (33bd71ab5d9248e59d8ad192b6cbc619): starting batch 2
24/11/23 17:37:23 FUNCTION LOG (ff78ff43033642eea84ddf083638e4b6): done
24/11/23 17:37:24 FUNCTION LOG (33bd71ab5d9248e59d8ad192b6cbc619): done


Wait 30s here

In [23]:
[[[subjob.result.get() for subjob in job.subjobs] for job in job.subjobs] for job in job.subjobs]

[[[3]], [[2]]]